In [65]:
import docx

def make_questions(path):
    #read docx file
    doc = docx.Document(path)
    # Danh sách chứa các câu hỏi
    questions = []
    current_question = ""
    # Lặp qua từng đoạn văn trong file DOCX
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        # Nếu bắt đầu bằng "Câu", đó là câu hỏi mới
        if text.startswith("Câu"):
            if current_question:  # Nếu đã có câu hỏi trước đó, thêm vào danh sách
                questions.append([current_question[7:]])
            current_question = text  # Bắt đầu câu hỏi mới
        elif current_question:  # Nếu không phải câu hỏi, tiếp tục thêm đáp án
            current_question += " " + text

    # Thêm câu hỏi cuối cùng (nếu có)
    if current_question:
        questions.append([current_question.strip()])
    return questions

In [58]:
from docx import Document
# Path to the DOCX file
def make_answers(path):
    # Open the Word document
    document = Document(path)

    # List to store underlined text
    underlined_text = []
    current_underlined = ""  # To handle multi-line underlined text

    # Loop through all paragraphs and their runs
    for paragraph in document.paragraphs:
        for run in paragraph.runs:
            if run.underline:  # Check if the run is underlined
                current_underlined += run.text.strip() + ""  # Append text and keep it continuous
            else:
                if current_underlined:  # If switching from underlined to non-underlined text
                    underlined_text.append(current_underlined.strip())
                    current_underlined = ""  # Reset the temporary variable

    # Add any remaining underlined text
    if current_underlined:
        underlined_text.append(current_underlined.strip())

    # Return the list of underlined text
    return underlined_text


In [95]:
path = "D:/533463523-NGAN-HANG-CAU-HỎI-TRẮC-NGHIỆM-MON-CHỦ-NGHĨA-XA-HỘI-KHOA-HỌC.docx"
questions = make_questions(path)
answers = make_answers(path)
# make csv file have 2 column "cau hoi" and "dap an"
import csv
with open('test-data\chu-nghia-xa-hoi-khoa-hoc-questions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "Answer"])
    for i in range(len(questions)):
        writer.writerow([questions[i][0], answers[i]])

<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\APTS\AppData\Local\Temp\ipykernel_17312\437966213.py:6: SyntaxWarning: invalid escape sequence '\c'
  with open('test-data\chu-nghia-xa-hoi-khoa-hoc-questions.csv', 'w', newline='') as file:


In [61]:
len(questions),len(answers)

(4, 0)

In [66]:
questions= make_questions(path)

In [92]:
import re

# Đọc nội dung từ file DOCX
doc = docx.Document("D:/pldc.docx")
text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
# Tách câu hỏi dựa trên từ khóa "Câu " và "Đáp án"
questions = re.split(r"(?=Câu \d+\. .*?)", text)

# Xử lý từng câu hỏi
data = []
for question in questions:
    if question.strip():  # Bỏ qua các đoạn trống
        match = re.search(r"(Câu \d+\. .*?)", question, re.DOTALL)
        if match:
            full_question = match.group(1).strip()
            correct_answer = match.group(2).strip()
            data.append({"Question": full_question})

# Lưu dữ liệu vào file CSV
import csv
with open("test-data\Phap_Luat_Trac_Nghiem.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Question"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)
print("Dữ liệu đã được lưu vào file CSV.")


Dữ liệu đã được lưu vào file CSV.


<>:21: SyntaxWarning: invalid escape sequence '\P'
<>:21: SyntaxWarning: invalid escape sequence '\P'
C:\Users\APTS\AppData\Local\Temp\ipykernel_17312\77413453.py:21: SyntaxWarning: invalid escape sequence '\P'
  with open("test-data\Phap_Luat_Trac_Nghiem.csv", "w", newline="", encoding="utf-8") as file:


In [134]:
from docx import Document
import csv
import re

# Đường dẫn file
input_docx = "D:/tthcm.docx"  # Thay bằng đường dẫn thực tế
output_csv = "test-data\Tu_Tuong_HCM_Trac_Nghiem.csv"  # File xuất CSV

# Mở file DOCX
document = Document(input_docx)

# Biến lưu trữ
data = []
current_question = ""
answers = []

# Hàm kiểm tra xem đoạn văn có thuộc danh sách không
def is_list_item(paragraph):
    return paragraph.style.name.startswith("List")  # Kiểm tra nếu paragraph thuộc danh sách

# Đọc từng đoạn văn trong file DOCX
for paragraph in document.paragraphs:
    text = paragraph.text.strip()

    if is_list_item(paragraph):
        # Kiểm tra nếu là đáp án (bắt đầu bằng a), b), ...)
        if re.match(r"^[a-d]\)\s", text):
            answers.append(text)
        elif text:  # Coi là câu hỏi nếu không khớp với định dạng đáp án
            # Lưu câu hỏi trước đó (nếu có)
            if current_question and answers:
                correct_answer = next((a.replace("(đ)", "").strip() for a in answers if "(đ)" in a), "")
                data.append([current_question.strip(), correct_answer])
                answers = []  # Reset danh sách đáp án
            current_question = text  # Cập nhật câu hỏi mới
    elif text:  # Đoạn văn không thuộc danh sách (xử lý câu hỏi độc lập)
        if current_question and answers:
            correct_answer = next((a.replace("(đ)", "").strip() for a in answers if "(đ)" in a), "")
            data.append([current_question.strip(), correct_answer])
            current_question = ""
            answers = []

# Xử lý câu hỏi cuối cùng
if current_question and answers:
    correct_answer = next((a.replace("(đ)", "").strip() for a in answers if "(đ)" in a), "")
    data.append([current_question.strip(), correct_answer])

# Ghi dữ liệu vào file CSV
with open(output_csv, mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "Answer"])  # Tiêu đề cột
    writer.writerows(data)

print(f"Dữ liệu đã được tách và lưu vào {output_csv}.")


<>:7: SyntaxWarning: invalid escape sequence '\T'
<>:7: SyntaxWarning: invalid escape sequence '\T'
C:\Users\APTS\AppData\Local\Temp\ipykernel_17312\4145004070.py:7: SyntaxWarning: invalid escape sequence '\T'
  output_csv = "test-data\Tu_Tuong_HCM_Trac_Nghiem.csv"  # File xuất CSV


Dữ liệu đã được tách và lưu vào test-data\Tu_Tuong_HCM_Trac_Nghiem.csv.


In [131]:
answers

[]